<a href="https://colab.research.google.com/github/basugautam/Reproducibility-Challenge-Project/blob/Architecture-Files/14_Dynamic_Constraint_Adaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load dataset from local Windows path (if running on local machine)
file_path = r'C:\LCTSF\Dataset\timeseries_data.csv'  # Use raw string (r'path') to avoid escape issues
df = pd.read_csv(file_path)

# Display first few rows
print(df.head())


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\LCTSF\\Dataset\\timeseries_data.csv'

In [ ]:
from google.colab import files
uploaded = files.upload()  # Manually upload timeseries_data.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load from Google Drive (modify path accordingly)
df = pd.read_csv('/content/drive/MyDrive/LCTSF/Dataset/timeseries_data.csv')
print(df.head())


In [ ]:
import numpy as np

# Function to adapt constraints dynamically based on previous forecasting errors
def dynamic_constraints(errors, step):
    """ Adjusts constraints based on error patterns over time steps """
    if step < len(errors) // 3:
        return np.max(errors) * 0.8  # Stricter constraints for early steps
    elif step < 2 * len(errors) // 3:
        return np.max(errors) * 0.9  # Moderate constraints
    else:
        return np.max(errors) * 1.2  # Looser constraints for later steps


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Extract features and target
df['lag_1'] = df['value'].shift(1)  # Simple lag feature
df.dropna(inplace=True)

X = df[['lag_1']]  # Features
y = df['value']    # Target

# Train-test split
train_size = int(len(df) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Compute initial error
errors = np.abs(y_test - y_pred)

# Apply dynamic constraint adaptation
adjusted_constraints = [dynamic_constraints(errors, step) for step in range(len(y_test))]

# Print dynamic constraints
print("Dynamic Constraints:", adjusted_constraints[:10])


In [ ]:
from scipy.optimize import minimize

# Define an objective function that penalizes high errors dynamically
def objective(params, X, y, constraints):
    forecasted_values = np.dot(X, params)
    error = np.abs(y - forecasted_values)

    # Apply penalties for exceeding dynamic constraints
    penalty = np.sum(np.maximum(error - constraints, 0) ** 2)

    return np.sum(error) + penalty  # Minimize error while respecting constraints

# Initial model parameters
initial_params = np.ones(X_train.shape[1])

# Optimize with dynamic constraints
result = minimize(objective, initial_params, args=(X_train, y_train, adjusted_constraints), method='SLSQP')

# Get optimized model parameters
optimized_params = result.x
print("Optimized Model Parameters:", optimized_params)


In [ ]:
# Forecast using optimized parameters
optimized_forecast = np.dot(X_test, optimized_params)

# Compute final error
final_rmse = np.sqrt(np.mean((y_test - optimized_forecast) ** 2))
print(f"Final RMSE with Dynamic Constraints: {final_rmse}")
